# Advent of code 2020: Day 20

## Import

In [17]:
import math as M
import collections as C
import itertools as I
from helper import *


day = 20
data = get_input_groups(day, 2020)

In [18]:
def as_cols(grid: Iterable[Iterable[Any]],) -> list[list[Any]]:
    return [list(x) for x in zip(*grid)]

## Common

In [19]:
tileset = data
tiles = {}
borders = {}  # Top Bottom Left Right, flipped
for tile in tileset:
    n = ints(tile[0])[0]
    tiles[n] = tile[1:]
    bor = [tile[1], tile[-1]]
    tile = ["".join(x) for x in as_cols(tile[1:])]
    bor += [tile[0], tile[-1]]
    bor += [x[::-1] for x in bor]
    borders[n] = bor

## Part 1

This probably shouldn't work and idk what I was thinking when I wrote it, but it does

In [20]:
size = int(M.sqrt(len(tiles)))
m = []
counts = C.Counter()

for tile in borders:
    for side in borders[tile]:
        for other in [other for other in borders if other != tile]:
            for otherside in borders[other]:
                if otherside == side:
                    counts[tile] += 1

t = 1
for c in counts:
    if counts[c] == 4:
        t *= c
print(t)

29584525501199

## Better Part 2

In [21]:
def get_edges(tile):
    cols = as_cols(tile)
    return [tile[0], tile[-1][::-1], "".join(cols[0])[::-1], "".join(cols[-1])]  # Top bottom left right

def get_rotations(tile):
    rots = [tile, rotated(tile)]
    rots += [rot[::-1] for rot in rots]
    rots += [[x[::-1] for x in rot] for rot in rots]
    return rots

def rotated(tile):
    return ["".join(e)[::-1] for e in as_cols(tile)]

def simplify_edge(edge):
    return min(edge, edge[::-1])

In [22]:
edge_use_count = C.Counter()
edge_to_tiles = {}
rotations = {}
for number, tile in tiles.items():
    rotations[number] = get_rotations(tile)
    for edge in get_edges(tile):
        edge = simplify_edge(edge)
        edge_use_count[edge] += 1
        if edge in edge_to_tiles:
            edge_to_tiles[edge].add(number)
        else:
            edge_to_tiles[edge] = {number}
    

In [23]:
corners = []
for c in counts:
    if counts[c] == 4:
        corners += [c]

Top row

In [24]:
used = {corners[0]}
arrangement_tiles = [[tiles[corners[0]]]]
arrangement_numbers = [[corners[0]]]

for x in range(1, size):
    last_tile = arrangement_tiles[0][x-1]
    last_number = arrangement_numbers[0][x-1]
    right_edge = get_edges(last_tile)[3]
    next_number = [n for n in edge_to_tiles[simplify_edge(right_edge)] if n not in used][0]
    
    for rot in rotations[next_number]:
        left_edge = get_edges(rot)[2][::-1]
        if left_edge == right_edge:
            arrangement_tiles[0] += [rot]
            arrangement_numbers[0] += [next_number]
            used.add(next_number)
            break

Rest of the rows

In [25]:
for y in range(1, size):
    row_tiles = []
    row_numbers = []
    for x in range(size):
        above_tile = arrangement_tiles[y-1][x]
        above_number = arrangement_numbers[y-1][x]
        bottom_edge = get_edges(above_tile)[1]
        next_number = [n for n in edge_to_tiles[simplify_edge(bottom_edge)] if n not in used][0]
        
        for rot in rotations[next_number]:
            top_edge = get_edges(rot)[0][::-1]
            if bottom_edge == top_edge:
                row_tiles += [rot]
                row_numbers += [next_number]
                used.add(next_number)
                break
    arrangement_tiles += [row_tiles]
    arrangement_numbers += [row_numbers]

Find the sea monsters

In [26]:
arrangement_string_lines = []
for tile_row in arrangement_tiles:
    arrangement_string_lines += ["".join(tile[y][1:-1] for tile in tile_row) for y in range(1, len(tile_row[0]) - 1)]

arrangement_rotations = get_rotations(arrangement_string_lines)
monster = """\
                  #
#    ##    ##    ###
 #  #  #  #  #  #""".split('\n')

def find_monsters(arrangement_string_lines):
    c = 0
    for y, row in enumerate(arrangement_string_lines):
        for x, cell in enumerate(row):
            found = True
            for my, mrow in enumerate(monster):
                for mx, mcell in enumerate(mrow):
                    if mcell != "#":
                        continue
                    ny, nx = y + my, x + mx
                    if not(ny < len(arrangement_string_lines) and nx < len(row)) or arrangement_string_lines[y + my][x + mx] != "#":
                        found = False
                        break
                else:
                    continue
                break
            if found:
                c += 1
    return c

for rot in arrangement_rotations:
    if find_monsters(rot):
        print("".join(rot).count("#") - "".join(monster).count("#") * find_monsters(rot))
        break

1665

## Old Part 2 Shennanigans
_This all deserves to be deleted, but I'll let it live for now_

In [27]:
corners = []
for c in counts:
    if counts[c] == 4:
        corners += [c]

links = {}
for tile in borders:
    for side in borders[tile]:
        for other in [other for other in borders if other != tile]:
            for otherside in borders[other]:
                if otherside == side:
                    if tile in links:
                        links[tile][side] = other
                    else:
                        links[tile] = {side: other}
# print(links)

In [28]:
def get_adjacents(links: dict, side: int):
    a = set()
    for other in links[side]:
        a.add(links[side][other])
    return a
    

In [29]:
arrangement = [[None,] * size for x in range(size)]
used = set()

dirs = [(1, 0), (0, 1), (-1, 0), (0, -1)]

def valid(x, y):
    return 0 <= x < size and 0 <= y < size and arrangement[y][x] != None

arrangement[0][0] = corners[0]
for y in range(size):
    for x in range(size):
        if x == y == 0:
            continue
        adjacent = set()
        for di in dirs:
            nx, ny = x + di[0], y + di[1]
            if valid(nx, ny):
                adjacent.add(arrangement[ny][nx])
        
        for possible in [possible for possible in links if possible not in used]:
            if all(k in get_adjacents(links, possible) for k in adjacent):
                pass

Literally started generating images and a graph layout to use with yEd... gave up and went back to coding it. Might make a visualization later

In [30]:
do_stupid_things = False

In [31]:
if do_stupid_things:
    import pyyed

    yed = pyyed.Graph()

    nodes = set()
    for number in tiles:
        number = str(number)
        if number not in nodes:
            nodes.add(number)
            yed.add_node(number)
        for adjacent in get_adjacents(links, int(number)):
            adjacent = str(adjacent)
            if adjacent not in nodes:
                nodes.add(adjacent)
                yed.add_node(adjacent)
            yed.add_edge(number, adjacent)
            
    yed.write_graph("full.graphml", pretty_print=True)


In [32]:
if do_stupid_things:
    from PIL import Image

    for number in tiles:
        fname = f"20_tiles/{number}.png"
        out = Image.new("RGB", (10, 10))
        r = []
        for y, row in enumerate(tiles[number]):
            # print(len(row))
            for x, c in enumerate(row):
                if c == "#":
                    r.append((0, 0, 0))
                else:
                    r.append((255, 255, 255))
        # print(len(r))
        out.putdata(r)
        out.save(fname)